<a href="https://colab.research.google.com/github/Doumbaly/Detection-de-Pose-Visage/blob/main/D%C3%A9tection_de_Pose_et_Visage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Détection de Pose et boîte délimitant le visage

La détection de pose est un domaine d'étude actif dans le domaine de la vision par ordinateur.
Dans ce test nous allons couvrir l'une de ces applications de détection et d'estimation de pose à l'aide de l'apprentissage automatique et de certaines des bibliothèques très utiles en python(OpenCV,Mediapipe).

L'estimation de pose est une technique de vision par ordinateur pour suivre les mouvements d'une personne ou d'un objet. Ceci est généralement effectué en trouvant l'emplacement des points clés pour les objets donnés.

1- Ecrire un programme python qui va générer une vidéo de sortie et produit les informations sous forme de tableau dans un fichier excel

### Bibliothèques utilisées

In [ ]:
!pip install opencv-contrib-python

In [ ]:
#!pip install mediapipe opencv-python pandas scikit-learn

In [ ]:
!pip install mediapipe opencv-python

In [ ]:
!pip install ffmpeg-python

In [ ]:
import mediapipe as mp
import cv2
from matplotlib import pyplot as plt

In [ ]:
# Initialisation des classes
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [ ]:
mpFaceDetection = mp.solutions.face_detection
mpDraw = mp.solutions.drawing_utils
faceDetection = mpFaceDetection.FaceDetection(0.75)

## Le modèle

In [ ]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import time

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
count = 0
alldata = []
fps_time = 0
 
 
pose_corps = ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER', 'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT',
              'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB',
              'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']
 
pose_main = ['WRIST', 'THUMB_CPC', 'THUMB_MCP', 'THUMB_IP', 'THUMB_TIP', 'INDEX_FINGER_MCP', 'INDEX_FINGER_PIP', 'INDEX_FINGER_DIP', 'INDEX_FINGER_TIP', 'MIDDLE_FINGER_MCP',
               'MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_DIP', 'MIDDLE_FINGER_TIP', 'RING_FINGER_PIP', 'RING_FINGER_DIP', 'RING_FINGER_TIP',
               'RING_FINGER_MCP', 'PINKY_MCP', 'PINKY_PIP', 'PINKY_DIP', 'PINKY_TIP']
 
pose_main_2 = ['WRIST2', 'THUMB_CPC2', 'THUMB_MCP2', 'THUMB_IP2', 'THUMB_TIP2', 'INDEX_FINGER_MCP2', 'INDEX_FINGER_PIP2', 'INDEX_FINGER_DIP2', 'INDEX_FINGER_TIP2', 'MIDDLE_FINGER_MCP2',
               'MIDDLE_FINGER_PIP2', 'MIDDLE_FINGER_DIP2', 'MIDDLE_FINGER_TIP2', 'RING_FINGER_PIP2', 'RING_FINGER_DIP2', 'RING_FINGER_TIP2',
               'RING_FINGER_MCP2', 'PINKY_MCP2', 'PINKY_PIP2', 'PINKY_DIP2', 'PINKY_TIP2']
 
cap = cv2.VideoCapture('exercising.mp4')
suc,frame_video = cap.read()
vid_writer = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_video.shape[1], frame_video.shape[0]))
with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            #Utiliser 'break' si nous avons une video sinon 'continue'.
            #continue
            break
 
        # Retournez l'image horizontalement pour un affichage ultérieur en vue selfie et convertion
        # BGR image à RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # Pour ameliorer les performances, marquons image.flags.writeable=false
        #image.flags.writeable??
        image.flags.writeable = False
        results = holistic.process(image)
 
        # Annotation de points de repères sur l'image.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image_asli = np.copy(image)
        image = np.zeros(image.shape)
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,32), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(245,66,200), thickness=1, circle_radius=1))
        
          # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,12,3), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,12), thickness=1, circle_radius=1)
                                 )
 
      #  Si None:
        if results.pose_landmarks: 
            data_corps = {}
            for i in range(len(pose_corps)):
                results.pose_landmarks.landmark[i].x = results.pose_landmarks.landmark[i].x * image.shape[0]
                results.pose_landmarks.landmark[i].y = results.pose_landmarks.landmark[i].y * image.shape[1]
                data_corps.update(
                {pose_corps[i] : results.pose_landmarks.landmark[i]}
                )
            alldata.append(data_corps)
        
        if results.face_landmarks: 
            data_corps = {}
            for i in range(len(pose_corps)):
                results.face_landmarks.landmark[i].x = results.face_landmarks.landmark[i].x * image.shape[0]
                results.face_landmarks.landmark[i].y = results.face_landmarks.landmark[i].y * image.shape[1]
                data_corps.update(
                {pose_corps[i] : results.face_landmarks.landmark[i]}
                )
            alldata.append(data_corps)
        
        
        
        if results.right_hand_landmarks:
            data_main_droite = {}
            for i in range(len(pose_main)):
                results.right_hand_landmarks.landmark[i].x = results.right_hand_landmarks.landmark[i].x * image.shape[0]
                results.right_hand_landmarks.landmark[i].y = results.right_hand_landmarks.landmark[i].y * image.shape[1]
                data_corps.update(
                {pose_main[i] : results.right_hand_landmarks.landmark[i]}
                )
            alldata.append(data_corps)
 
        if results.left_hand_landmarks:
            data_main_gauche  = {}
            for i in range(len(pose_main)):
                results.left_hand_landmarks.landmark[i].x = results.left_hand_landmarks.landmark[i].x * image.shape[0]
                results.left_hand_landmarks.landmark[i].y = results.left_hand_landmarks.landmark[i].y * image.shape[1]
                data_corps.update(
                {pose_main_2[i] : results.left_hand_landmarks.landmark[i]}
                )
            alldata.append(data_corps)
 
 
        #cv2.namedWindow('MediaPipe Holistic', cv2.WND_PROP_qFULLSCREEN)
        #cv2.setWindowProperty('MediaPipe Holistic', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.putText(image, "FPS: %f" % (1.0 / (time.time() - fps_time)), (10, 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2,)
        cv2.imshow('MediaPipe Holistic', image)
        cv2.imshow('Gambar asli', image_asli)
        count = count + 1
        #print(count)
        fps_time = time.time()
        vid_writer.write(image)
        #plt.imshow((image*255).astype(np.uint8))
        #plt.savefig("image-frame/" + str(count) + ".jpg")
        if cv2.waitKey(5) & 0xFF == ord('q'):
            df = pd.DataFrame(alldata)
            df.to_excel("point_repere.xlsx")
            break
cap.release()


Ignoring empty camera frame.


In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
count = 0
alldata = []
fps_time = 0

## Les coordonnées de pose, du visage et des mains

In [ ]:
results.face_landmarks # de repere du visage
#results.face_landmarks.landmark

In [ ]:
# Pose repères
results.pose_landmarks.landmark

[x: 304.8069152832031
y: 116.36994171142578
z: -0.48211750388145447
visibility: 0.9999475479125977
, x: 306.9285888671875
y: 110.58177947998047
z: -0.42382392287254333
visibility: 0.999961256980896
, x: 313.53631591796875
y: 110.66045379638672
z: -0.4239772856235504
visibility: 0.9999653100967407
, x: 320.0529479980469
y: 110.84622192382812
z: -0.42376285791397095
visibility: 0.9999665021896362
, x: 284.4000549316406
y: 111.42975616455078
z: -0.4532322883605957
visibility: 0.9999492168426514
, x: 272.45538330078125
y: 112.11679077148438
z: -0.45317551493644714
visibility: 0.9999443292617798
, x: 260.3934326171875
y: 112.96100616455078
z: -0.4535560607910156
visibility: 0.9999357461929321
, x: 319.6290283203125
y: 116.6854248046875
z: -0.06376425921916962
visibility: 0.9999656677246094
, x: 235.7295379638672
y: 119.58637237548828
z: -0.20681503415107727
visibility: 0.9999576210975647
, x: 320.777099609375
y: 126.08522033691406
z: -0.34663110971450806
visibility: 0.9999194145202637
, x: 

In [ ]:
# Main droite repères
results.right_hand_landmarks.landmark

[x: 172.09449768066406
y: 219.62693786621094
z: -1.5830957522666722e-07
, x: 152.7646026611328
y: 214.26272583007812
z: -0.010820234194397926
, x: 139.85403442382812
y: 205.78402709960938
z: -0.02393254078924656
, x: 138.7690887451172
y: 198.01837158203125
z: -0.03535022214055061
, x: 147.0047607421875
y: 192.92318725585938
z: -0.04341942071914673
, x: 142.13572692871094
y: 208.74014282226562
z: -0.03828785941004753
, x: 149.13739013671875
y: 196.17489624023438
z: -0.058234356343746185
, x: 150.09942626953125
y: 198.69297790527344
z: -0.0630224421620369
, x: 149.59283447265625
y: 202.66041564941406
z: -0.06233195587992668
, x: 161.6962890625
y: 209.8805389404297
z: -0.04087994620203972
, x: 168.9125213623047
y: 195.98223876953125
z: -0.058351192623376846
, x: 168.1522979736328
y: 198.98779296875
z: -0.05470721051096916
, x: 167.8394775390625
y: 203.79200744628906
z: -0.0494922436773777
, x: 180.80735778808594
y: 210.32061767578125
z: -0.04365293309092522
, x: 185.81716918945312
y: 197.